In [ ]:
# @title ▶️ Запустить приложение (надёжный запуск для Colab)
import os, sys, subprocess
def _ensure_deps():
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-U', 'pandas==2.2.2', 'openpyxl>=3.1,<4', 'gradio>=4.0'])
def _cd_to_project():
    if os.path.isdir('/content/tpok003'):
        os.chdir('/content/tpok003')
        return '/content/tpok003'
    return os.getcwd()
def _install_from_project():
    project_dir = '/content/tpok003'
    target = project_dir if os.path.exists(os.path.join(project_dir, 'pyproject.toml')) else os.getcwd()
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-e', target])
    return target
_ensure_deps()
proj_dir = _cd_to_project()
installed_from = _install_from_project()
print('Working dir:', os.getcwd())
print('yourpkg installed from:', installed_from)
from yourpkg.colab_app import launch_app
launch_app()
